# 1. Import Library

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


# 2. Set Random Seed

In [2]:
random_seed = 2017
np.random.seed(random_seed)
tf.set_random_seed(random_seed)

# 3. Load MNIST Data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

print('number of MNIST train data : %5d' % len(mnist.train.images))
print('number of MNIST test data  : %5d' % len(mnist.test.images))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
number of MNIST train data : 55000
number of MNIST test data  : 10000


# 4. Data Augmentation
- Rotation (Range = 10, 20, 30)
- Width Shift (Range = 0.1, 0.2, 0.3)

In [4]:
augmentation_size = 440000

images = np.concatenate((mnist.train.images.reshape(-1, 28, 28, 1), mnist.validation.images.reshape(-1, 28, 28, 1)), axis=0)
labels = np.concatenate((mnist.train.labels, mnist.validation.labels), axis=0)

datagen_list = [
                ImageDataGenerator(rotation_range=10),
                ImageDataGenerator(rotation_range=20),
                ImageDataGenerator(rotation_range=30),
                ImageDataGenerator(width_shift_range=0.1),
                ImageDataGenerator(width_shift_range=0.2),
                ImageDataGenerator(width_shift_range=0.3),
               ]

for datagen in datagen_list:
    datagen.fit(images)
    for image, label in datagen.flow(images, labels, batch_size=augmentation_size, shuffle=True, seed=random_seed):
        images = np.concatenate((images, image), axis=0)
        labels = np.concatenate((labels, label), axis=0)
        break
        
images = images.reshape(-1, 784)        

In [5]:
print('final number of train data : %7d' % len(images))
print('images shape : ', images.shape)
print('labels shape : ', labels.shape)

final number of train data : 1800000
images shape :  (1800000, 784)
labels shape :  (1800000, 10)


# 5. Parameter

In [6]:
epochs = 30
conv_filters = [64, 128, 128, 128, 128]
conv_kernerls = [3, 3]
dens_units= [1024, 1024, 10]
keep_prob = 0.5
learning_rate = 0.0001
batch_size = 256

conv_active_fn = tf.nn.elu
dens_active_fn = tf.nn.elu

# 6. Modeling

### Placeholder

In [7]:
input_data = tf.placeholder(tf.float32, shape=[None, 784])
label_data = tf.placeholder(tf.float32, shape=[None, 10])
train_mode = tf.placeholder(tf.bool)

### Convolutional Layers

In [8]:
conv1_1 = tf.layers.conv2d(inputs=tf.reshape(input_data, [-1, 28, 28, 1]), filters=conv_filters[0], kernel_size=conv_kernerls, padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
conv1_2 = tf.layers.conv2d(inputs=conv1_1, filters=conv_filters[1], kernel_size=conv_kernerls, padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
pool1 = tf.layers.max_pooling2d(inputs=conv1_2, pool_size=[2, 2], padding='valid', strides=2)
dropout1 = tf.layers.dropout(inputs=pool1, rate=keep_prob, training=train_mode)

conv2_1 = tf.layers.conv2d(inputs=dropout1, filters=conv_filters[2], kernel_size=conv_kernerls, padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
conv2_2 = tf.layers.conv2d(inputs=conv2_1, filters=conv_filters[3], kernel_size=conv_kernerls, padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
pool2 = tf.layers.max_pooling2d(inputs=conv2_2, pool_size=[2, 2], padding='valid', strides=2)
dropout2 = tf.layers.dropout(inputs=pool2, rate=keep_prob, training=train_mode)

conv3 = tf.layers.conv2d(inputs=dropout2, filters=conv_filters[4], kernel_size=conv_kernerls, padding='SAME', activation=conv_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d())
pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2, 2], padding='valid', strides=2)
dropout3 = tf.layers.dropout(inputs=pool3, rate=keep_prob, training=train_mode)

### Fully Connected Layers

In [9]:
flat = tf.reshape(dropout3, [-1, dropout3.shape[1].value*dropout3.shape[2].value*dropout3.shape[3].value])
dense1 = tf.layers.dense(inputs=flat, units=dens_units[0],activation=dens_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer())
dropout4 = tf.layers.dropout(inputs=dense1, rate=keep_prob, training=train_mode)
dense2 = tf.layers.dense(inputs=dropout4, units=dens_units[1],activation=dens_active_fn, kernel_initializer=tf.contrib.layers.xavier_initializer())
dropout5 = tf.layers.dropout(inputs=dense2, rate=keep_prob, training=train_mode)
logits = tf.layers.dense(inputs=dropout5, units=dens_units[2],activation=tf.nn.softmax, kernel_initializer=tf.contrib.layers.xavier_initializer())

### Define Cost_fn, Optimizer

In [10]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label_data))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(label_data, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 7. Training

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
start_time = datetime.now()
print('Training Start : %s\n' % start_time)
for epoch in range(epochs):
    epoch_start_time = datetime.now()
    total_batch = int(len(images) / batch_size)
    for batch in range(total_batch):
        batch_xs, batch_ys = images[batch * batch_size : (batch + 1) * batch_size], labels[batch * batch_size : (batch + 1) * batch_size]
        _, loss, train_acc = sess.run([optimizer, cost, accuracy], feed_dict={input_data: batch_xs, label_data: batch_ys, train_mode: True})
    print('Epoch : %2d | Train Accuracy : %f | Epoch Time : %s' % (epoch, train_acc, datetime.now() - epoch_start_time))
end_time = datetime.now()
print('\nTraining Finish : %s\n' % (end_time))

Training Start : 2017-05-01 13:30:00.632804



# 8. Testing

In [ ]:
test_acc = 0
for i in range(20):
    batch_x, batch_y = mnist.test.next_batch(500)
    test_acc += np.equal(sess.run(tf.argmax(logits,1), feed_dict={input_data: batch_x, train_mode: False}), np.argmax(batch_y, 1)).sum()
print('Test Accuracy : %.4f' % (test_acc / 10000))